In [ ]:
pip install kaggle

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = r'C:\Users\nibha\.kaggle'


In [ ]:
!kaggle datasets download -d ronakbokaria/myntra-products-dataset


Dataset URL: https://www.kaggle.com/datasets/ronakbokaria/myntra-products-dataset
License(s): CC0-1.0
 98% 112M/115M [00:04<00:00, 34.0MB/s]
100% 115M/115M [00:04<00:00, 27.2MB/s]


In [ ]:
import zipfile
import pandas as pd

df = pd.read_csv('myntra.csv')  # Update with the correct CSV file name if different
#Data Preprocessing
df.drop(columns=['id', 'asin', 'img', 'purl'], inplace=True)







In [ ]:
df.columns


Index(['name', 'price', 'mrp', 'rating', 'ratingTotal', 'discount', 'seller'], dtype='object')

In [ ]:
df.isnull().sum()
# Handle missing values
df['name'].fillna('Unknown', inplace=True)
df['price'].fillna(df['price'].mean(), inplace=True)
df['rating'].fillna(df['rating'].mean(), inplace=True)


<ipython-input-7-42ca18de43e1>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['name'].fillna('Unknown', inplace=True)
<ipython-input-7-42ca18de43e1>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.me

In [10]:
import nltk


In [11]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [13]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [23]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Initialize the lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):

    if not isinstance(text, str):
        return text  # Return as-is if input is not a string

    # Convert to lowercase
    text = text.lower()

    # Remove non-alphabetic characters
    text = re.sub(r'[^a-z\s]', '', text)

    # Tokenize, remove stopwords, and apply lemmatization
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    # Return the processed text as a single string
    return ' '.join(tokens)

# Apply the preprocess_text function to the DataFrame columns
df['name'] = df['name'].apply(preprocess_text)
df['seller'] = df['seller'].apply(preprocess_text)


In [24]:
from sklearn.preprocessing import MinMaxScaler,LabelEncoder

scaler = MinMaxScaler()
df[['price', 'mrp']] = scaler.fit_transform(df[['price', 'mrp']])


In [25]:
def categorize_discount(discount):
    if discount > 70:
        return 'High'
    elif discount > 50:
        return 'Medium'
    else:
        return 'Low'

df['discount_category'] = df['discount'].apply(categorize_discount)


In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
df['discount_category'] = df['discount'].apply(categorize_discount)
label_encoder = LabelEncoder()
df['seller_encoded'] = label_encoder.fit_transform(df['seller'])
df['discount_category_encoded'] = label_encoder.fit_transform(df['discount_category'])

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df['name'])

# Final Data
final_df = pd.concat([df, pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())], axis=1)

# Label Encoding for Seller and Discount Category

# The `final_df` is now ready for similarity calculation

In [29]:
print(final_df[numerical_cols].isnull().sum())

price          0
mrp            1
rating         0
ratingTotal    1
discount       1
dtype: int64


In [30]:
final_df = final_df.dropna(subset=numerical_cols)


In [33]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial.distance import jaccard

df[numerical_cols] = df[numerical_cols].fillna(0)  # Replace NaN with 0
# Or replace with mean/median:
# df[numerical_cols] = df[numerical_cols].fillna(df[numerical_cols].mean())

# Text Feature: 'name' (Cosine Similarity with TF-IDF)
tfidf_name = tfidf_vectorizer.fit_transform(df['name'])
text_sim = cosine_similarity(tfidf_name)

# Numerical Features: Normalize and Compute Euclidean/Manhattan
numerical_cols = ['price', 'mrp', 'rating', 'ratingTotal', 'discount']
scaler = MinMaxScaler()
numerical_data = scaler.fit_transform(df[numerical_cols])
numerical_sim = 1 / (1 + euclidean_distances(numerical_data))  # Invert distances for similarity


# Combine Similarities (Weighted Sum)
combined_sim = 0.5 * text_sim + 0.3 * numerical_sim


In [34]:
def get_similar_products(product_idx, cosine_sim, top_k=5):
    sim_scores = list(enumerate(cosine_sim[product_idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    similar_products = sim_scores[1:top_k+1]  # Skip the first one (itself)
    return similar_products

# Example Usage
product_idx = 0  # Index of the product you want to find similar items for
similar_products = get_similar_products(product_idx, combined_sim, top_k=5)
print(similar_products)


[(1700, 0.8000000000000003), (3494, 0.777707672466597), (5210, 0.6555015590547799), (5254, 0.6555015590547799), (3468, 0.654056156718781)]


In [35]:
import numpy as np

def evaluate_recommendations(true_relevant_items, recommended_items, top_k=5):
    """
    Evaluates the precision, recall, and F1 score at K for a given product recommendation.

    :param true_relevant_items: List of relevant items (indices) for a given product (ground truth)
    :param recommended_items: List of recommended items (indices) for the product
    :param top_k: Number of top recommendations to consider
    :return: Dictionary containing precision, recall, and F1 score at K
    """

    # Only consider the top K recommendations
    recommended_top_k = recommended_items[:top_k]

    # Calculate the number of relevant items in the top K recommendations
    relevant_in_top_k = len(set(true_relevant_items) & set(recommended_top_k))

    # Precision at K
    precision_at_k = relevant_in_top_k / top_k if top_k > 0 else 0

    # Recall at K
    recall_at_k = relevant_in_top_k / len(true_relevant_items) if len(true_relevant_items) > 0 else 0

    # F1 Score at K
    f1_at_k = (2 * precision_at_k * recall_at_k) / (precision_at_k + recall_at_k) if precision_at_k + recall_at_k > 0 else 0

    return {
        'precision_at_k': precision_at_k,
        'recall_at_k': recall_at_k,
        'f1_at_k': f1_at_k
    }

# Example: Assume that for product_idx=0, the true relevant items are known
true_relevant_items = [2, 5, 8, 12]  # List of relevant items for the given product index (ground truth)
recommended_items = [0, 2, 3, 5, 8, 12, 4, 1]  # Recommended items (top k) from the recommendation system

# Evaluate the recommendation
evaluation_metrics = evaluate_recommendations(true_relevant_items, recommended_items, top_k=5)

# Print the metrics
print("Precision at K:", evaluation_metrics['precision_at_k'])
print("Recall at K:", evaluation_metrics['recall_at_k'])
print("F1 Score at K:", evaluation_metrics['f1_at_k'])


Precision at K: 0.6
Recall at K: 0.75
F1 Score at K: 0.6666666666666665


In [ ]:
# prompt: Precision at K: 0.6
# Recall at K: 0.75
# F1 Score at K: 0.6666666666666665 come uo wuth a good explanatioon for this project

This project develops a product recommendation system using the Myntra product dataset.  The system combines text-based similarity (using TF-IDF vectorization of product names) and numerical feature similarity (considering price, MRP, rating, discount, etc.) to generate recommendations.

Here's a breakdown of the project and an interpretation of the evaluation metrics:

**Data Preprocessing:**

1. **Data Cleaning:** The code handles missing values in the dataset.  Missing product names are filled with "Unknown," while missing numerical values (price, rating) are imputed using the mean of the respective columns.  This ensures that the model can handle incomplete data without errors.

2. **Text Preprocessing:** Product names are converted to lowercase, cleaned of non-alphanumeric characters, and lemmatized (reduced to their base form) after removing stop words (common words like "the," "a," "is"). This prepares the text for effective similarity comparison.

3. **Numerical Feature Scaling:**  Numerical features like price and MRP are scaled using MinMaxScaler. This normalization ensures that features with larger magnitudes don't disproportionately influence the similarity calculations.  A similar scaling is applied to other numerical columns (rating, discount etc) before the similarity calculations are performed.

4. **Feature Engineering:**  A 'discount_category' feature is created based on the discount percentage, dividing products into "High," "Medium," and "Low" discount categories. This potentially improves the recommendation system's ability to capture products that have similar discount rates.

5. **Encoding Categorical Features:**  Categorical features such as 'seller' and 'discount_category' are label encoded. This transforms them into numerical representations, enabling their use in similarity calculations.

**Similarity Calculation:**

1. **Text Similarity:**  TF-IDF vectors are created for each product name. Cosine similarity is computed between these vectors to find similar products based on their descriptions.

2. **Numerical Similarity:** Euclidean distances are computed among the numerical features.  The distances are inverted (1 / (1 + distance)), transforming them into a similarity score where smaller distances translate into higher similarity.  This is because, smaller distance in this case means higher similarity.

3. **Combined Similarity:**  A weighted average of the text-based and numerical similarities are combined.  The weighting (0.5 for text and 0.3 for numerical) determines how much each factor influences the final recommendations.


**Recommendation and Evaluation:**


1. **Recommendation Generation:** The `get_similar_products` function retrieves the top K most similar products based on the combined similarity score.

2. **Evaluation Metrics:** The code evaluates the recommendation system using precision, recall, and F1-score at K (K=5 in this case):
    * **Precision@K = 0.6:** Out of the top 5 recommended products, 60% are actually relevant to the user.
    * **Recall@K = 0.75:** Out of all the truly relevant products, the system identified 75% within its top 5 recommendations.
    * **F1-Score@K = 0.667:**  This is the harmonic mean of precision and recall and provides a balanced measure of both. The F1 score indicates a reasonable balance between correctly identifying relevant products (precision) and capturing a good portion of all relevant products (recall).

**Explanation of Results:**

The relatively good recall (0.75) indicates that the recommendation system is capturing a good proportion of the relevant items. This could mean that the combined similarity measure, using a good weighting of text and numerical features, works reasonably well. However, the precision (0.6) is somewhat lower, indicating that some of the top 5 recommendations might not always be relevant. This might happen when highly similar numerical data is coupled with text that is not necessarily closely related. The F1-score reflects this trade-off.


**Possible Improvements:**

* **Hyperparameter Tuning:** Experiment with different weightings for text and numerical features, or explore other similarity measures.  Also different values of K could influence the result.
* **Additional Features:** Incorporate other features like brand, color, category, etc.
* **More Sophisticated Models:**  Explore other recommendation models like collaborative filtering or deep learning-based approaches.
* **Explore Different Similarity Metrics:** Other numerical distance metrics such as Manhattan distance or cosine similarity on numerical features might improve performance.

These modifications could potentially improve both precision and recall, leading to a more effective recommendation system.
